In [1]:
import json
import influxdb
import pandas as pd
import matplotlib.pyplot as plt
from odapi.formats import InfluxDB
from odapi.toolbox.psychro import Constants, Mix

In [2]:
with open("../../../../secrets/lht65.json") as fh:
    client = influxdb.InfluxDBClient(**json.load(fh))

In [3]:
q = client.query('SELECT "value" FROM /device_frmpayload_data_node_.*/ GROUP BY "device_name", "location"')

InfluxDBClientError: 401: {"error":"authorization failed"}

In [ ]:
df = InfluxDB.to_frame(q.raw)

In [ ]:
df.head()

In [ ]:
df["time"] = df["time"].dt.round("20T")

In [ ]:
df["serie_name"] = df["serie_name"].apply(lambda x: x.split("_")[-1])
df["location"] = df["location"].replace({'badroom-children': 'bathroom-children'})

In [ ]:
dc = df.pivot_table(index="time", columns=["serie_name", "location"], values="value")

In [ ]:
dc = dc[dc["temperature"]<300]

In [ ]:
dc = dc.interpolate()

In [ ]:
dc.head()

In [ ]:
T = dc["temperature"]
RH = dc["humidity"]

In [ ]:
axe = T.plot()
axe.legend(bbox_to_anchor=(1,1))
axe.grid()

In [ ]:
axe = RH.plot()
axe.legend(bbox_to_anchor=(1,1))
axe.grid()

In [ ]:
TK = T + Constants.T0K
pw = pd.DataFrame(Mix.pisow(TK.values , phi=RH.values/100.)/1000., columns = TK.columns, index = T.index)

In [ ]:
axe = Mix.plot()
_ = axe.plot(TK, pw, ".", markersize=3)

In [ ]:
df2 = pd.read_csv("Psychro_data.csv", header=[0,1], parse_dates=[0], index_col=0)
df2.head()

In [ ]:
df2 = df2.loc["2019-11-09 09:30:00":"2019-11-16 09:30:00",:]

In [ ]:
T = df2["node_temperature"]
RH = df2["node_humidity"]

In [ ]:
axe = T.plot()
axe.legend(bbox_to_anchor=(1,1))
axe.grid()

In [ ]:
axe = RH.plot()
axe.legend(bbox_to_anchor=(1,1))
axe.grid()

In [ ]:
TK = T + Constants.T0K
pw = pd.DataFrame(Mix.pisow(TK.values , phi=RH.values/100.)/1000., columns = TK.columns, index = T.index)

In [ ]:
axe = Mix.plot()
axe.plot(TK, pw, ".", markersize=3)